In [45]:
"""This module prepares the data, splits into train/test, and performs undersampling to balance the data set. 
Then it trains and compares these machine learning models:
- Logistic Regression
- Ridge Classifier
- Random Forest Classifier
- K-Neighbours Classifier
-"""

import sqlite3
import pandas as pd

try:
    database_path = "data/data.db"
    conn = sqlite3.connect(database_path)
    query = open('data/fetch_all.sql', 'r').read()
    df = pd.read_sql_query(query, conn)
except (sqlite3.Error, FileNotFoundError) as e:
    print(f"An error occurred: {e}")
finally:
    if 'conn' in locals():
        conn.close()

# Filter only rows with 'yes' or 'no' in the 'y' column
df = df[df['y'].isin(['no', 'yes'])]

# Drop specified columns
df = df.drop(columns=['client_id', 'account_id', 'campaign_id', 'outcome_id', 'previous', 'poutcome', 'month'])

# Binning day of the week into Early, Mid, and Late

bins = [0, 9, 19, 31]
labels = ['early', 'mid', 'late']
df['day_bin'] = pd.cut(df['day'], bins=bins, labels=labels, right=True)

# Binning age into common bins
age_bins = [0, 20, 30, 40, 50, 60, 100]
age_labels = ['<20', '20-29', '30-39', '40-49', '50-59', '60+']
df['age_bin'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

In [46]:

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Identify numerical and categorical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit and transform the categorical columns
encoded_features = encoder.fit_transform(df[categorical_columns])

# Create a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical columns
scaled_numeric_features = scaler.fit_transform(df[numerical_columns])

# Create a DataFrame with the scaled numerical features
scaled_numeric_df = pd.DataFrame(scaled_numeric_features, columns=numerical_columns)

# Concatenate the scaled numerical features and the encoded categorical features
df_enc = pd.concat([scaled_numeric_df, encoded_df], axis=1)

print(df_enc.head())

        age   balance       day  duration  campaign     pdays  \
0  0.078478 -0.472088 -1.323585  0.446598 -0.576829 -0.407218   
1  0.173040 -0.275697 -1.323585 -0.804230 -0.576829 -0.407218   
2  1.496912 -0.418920 -1.323585 -0.346234 -0.576829 -0.407218   
3  1.496912 -0.193951 -1.323585 -0.149950 -0.576829 -0.407218   
4  1.686036 -0.472753 -1.323585 -0.146102 -0.576829 -0.407218   

   job_blue-collar  job_entrepreneur  job_housemaid  job_management  ...  \
0              0.0               1.0            0.0             0.0  ...   
1              0.0               0.0            0.0             0.0  ...   
2              0.0               0.0            0.0             0.0  ...   
3              0.0               0.0            0.0             0.0  ...   
4              1.0               0.0            0.0             0.0  ...   

   marital_single  education_secondary  education_tertiary  education_unknown  \
0             0.0                  0.0                 1.0             

In [50]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming df_encoded_min_max_scaled is the dataframe to be split
X = df_enc.drop(columns=['y_yes'])
y = df_enc['y_yes']

# Apply SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [51]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets with 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (6400, 27)
X_test shape: (1600, 27)
y_train shape: (6400,)
y_test shape: (1600,)


In [52]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the models and their hyperparameters
models = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['liblinear']
        }
    },
    'Ridge Classifier': {
        'model': RidgeClassifierCV(alphas=[0.1, 1.0, 10.0], cv=5),
        'params': {}
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
    },
    'K-Neighbors Classifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    }
}

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and hyperparameter tuning
results = []
for model_name, model_info in models.items():
    if model_name == 'Ridge Classifier':
        clf = model_info['model']
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results.append({
            'Model': model_name,
            'Best Parameters': 'N/A (RidgeClassifierCV performs internal CV)',
            'Cross-Validation Score': clf.score(X_train, y_train),
            'Test Accuracy': accuracy,
            'Classification Report': classification_report(y_test, y_pred)
        })
    else:
        clf = GridSearchCV(model_info['model'], model_info['params'], cv=5, scoring='accuracy')
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results.append({
            'Model': model_name,
            'Best Parameters': clf.best_params_,
            'Cross-Validation Score': clf.best_score_,
            'Test Accuracy': accuracy,
            'Classification Report': classification_report(y_test, y_pred)
        })

# Display the results
for result in results:
    print(f"Model: {result['Model']}")
    print(f"Best Parameters: {result['Best Parameters']}")
    print(f"Cross-Validation Score: {result['Cross-Validation Score']}")
    print(f"Test Accuracy: {result['Test Accuracy']}")
    print(f"Classification Report:\n{result['Classification Report']}")
    print("\n" + "="*80 + "\n")

Model: Logistic Regression
Best Parameters: {'C': 0.1, 'solver': 'liblinear'}
Cross-Validation Score: 0.8084374999999999
Test Accuracy: 0.8
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.80      0.80       803
         1.0       0.80      0.80      0.80       797

    accuracy                           0.80      1600
   macro avg       0.80      0.80      0.80      1600
weighted avg       0.80      0.80      0.80      1600



Model: Ridge Classifier
Best Parameters: N/A (RidgeClassifierCV performs internal CV)
Cross-Validation Score: 0.8021875
Test Accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

         0.0       0.79      0.79      0.79       803
         1.0       0.79      0.79      0.79       797

    accuracy                           0.79      1600
   macro avg       0.79      0.79      0.79      1600
weighted avg       0.79      0.79      0.79      1600



Model: Random Fo